In [1]:
import requests
import json
import spotipy
import pandas as pd
from typing import List, Dict

In [2]:
import spotipy.util as util

# Lấy token để gọi API

In [3]:
# username = "Chrisnguyen"
username = 'ntnhan54'
# client_id= "ac6f1d431d774310a0761aef45b2160a"
client_id = '30852d0a0cbe4c978fd66b78c48a82a5'
# client_secret = "b967f892029e4a21a5f187a5d5b6c784"
client_secret = 'a1d544ff27cd41ceb3912352abab2109'

redirect_uri  = "http://localhost:7777/callback"

scope = 'playlist-read-collaborative'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [4]:
print(token)

BQCDEYvvM4Gkp2Zw1clt0EBQ_WiAgpdPyohfcCqYW7jh7DcTPJmd9NB53F0pNTk3_IWVyO0MzAsQBbbvI7wZC4OnEUSKFOBzyxgW_ibR9_GFTMq1DvKkI6g8n_mrbwAJEluq48Vc30mRCaLr_G5gcZ2SAaSUf_EzoB4DNlw


In [5]:
def get_id(track_name: str, token: str) -> str:
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': f'Bearer ' + token,
    }
    params = [
        ('q', track_name),
        ('type', 'track'),
    ]
    try:
        response = requests.get('https://api.spotify.com/v1/search', 
                    headers = headers, params = params, timeout = 5)
        json = response.json()
        first_result = json['tracks']['items'][0]
        track_id = first_result['id']
        return track_id
    except:
        return None

In [6]:
def get_features(track_id: str, token: str) -> dict:
    sp = spotipy.Spotify(auth=token)
    try:
        features = sp.audio_features([track_id])
        return features[0]
    except:
        return None

# Lấy danh sách các playlist do spotify tổng hợp

In [7]:
def get_all_playlist(username: str, token: str) -> List[Dict]:
    headers = {
        'Accept': 'application/json',\
        'Content-Type': 'application/json',\
        'Authorization': f'Bearer ' + token,\
    }
    
    offset   = 0
    limit    = 50
    total    = 100000 #max offset in spotify
    playlist = []
    
    while offset < total:
        params = [("limit", limit), ("offset", offset)]
        
        response = requests.get(f"https://api.spotify.com/v1/users/{username}/playlists", headers = headers, params=params)
        json_obj = response.json()
        
        if response.status_code != 200:
            print(response)
            continue
        
        playlist.extend(json_obj["items"])
        n = offset + len(json_obj["items"])

        print(f"Successfully get from playlist {offset} to {n}")
        offset = n
        
        if json_obj["total"] < total:
            total = json_obj["total"]
            
        if not "next" in json_obj or json_obj["next"] == "":
            break

    return playlist

In [8]:
spotify_playlist = get_all_playlist("spotify", token)

Successfully get from playlist 0 to 50
Successfully get from playlist 50 to 100
Successfully get from playlist 100 to 150
Successfully get from playlist 150 to 200
Successfully get from playlist 200 to 250
Successfully get from playlist 250 to 300
Successfully get from playlist 300 to 350
Successfully get from playlist 350 to 400
Successfully get from playlist 400 to 450
Successfully get from playlist 450 to 500
Successfully get from playlist 500 to 550
Successfully get from playlist 550 to 600
Successfully get from playlist 600 to 650
Successfully get from playlist 650 to 700
Successfully get from playlist 700 to 750
Successfully get from playlist 750 to 800
Successfully get from playlist 800 to 850
Successfully get from playlist 850 to 900
Successfully get from playlist 900 to 950
Successfully get from playlist 950 to 1000
Successfully get from playlist 1000 to 1050
Successfully get from playlist 1050 to 1100
Successfully get from playlist 1100 to 1150
Successfully get from playlist 

In [9]:
playlist_id = [s["id"] for s in spotify_playlist]    

In [10]:
with open("playlist.txt", "w") as f:
    for ids in playlist_id:
        f.write(ids + "\n")

In [11]:
len(playlist_id)

1412

# Lấy tất cả các tracks_id và artist_id trong các playlist 

In [80]:
def getItemByPlaylist(ids: str, token: str) -> dict:

    headers = {
            'Accept': 'application/json',\
            'Content-Type': 'application/json',\
            'Authorization': f'Bearer ' + token,\
        }
    playListDetail = {
        'track_ids': set(),
        'artist_ids': set()
    }
    response = requests.get(f"https://api.spotify.com/v1/playlists/{ids}/tracks", headers = headers)
    json_obj = response.json()
    for item in json_obj['items']:
        try:
            playListDetail['track_ids'].add(item["track"]['id'])
            playListDetail['artist_ids'].add(item["track"]["artists"][0]['id'])
        except:
            pass
    return playListDetail

In [81]:
with open('playlist.txt') as f:
    playList_ids = f.read()
playList_ids = playList_ids.split('\n')[:-1]

In [82]:
playListDetails = getItemByPlaylist(playList_ids[0], token)
for i, ids in enumerate(playList_ids[1:]):
    print(i, ids)
    detail = getItemByPlaylist(ids, token)
    playListDetails['track_ids'] |= detail['track_ids']
    playListDetails['artist_ids'] |= detail['artist_ids']
try:
    playListDetails['track_ids'].remove(None)
    playListDetails['artist_ids'].remove(None)
except:
    pass

0 37i9dQZF1DX0XUsuxWHRQd
1 37i9dQZF1DX1lVhptIYRda
2 37i9dQZF1DX10zKzsJ2jva
3 37i9dQZF1DX4JAvHpjipBk
4 37i9dQZF1DX4sWSpwq3LiO
5 37i9dQZF1DX4SBhb3fqCJd
6 37i9dQZF1DWXRqgorJj26U
7 37i9dQZF1DX4dyzvuaRJ0n
8 37i9dQZF1DXcF6B6QPhFDv
9 37i9dQZF1DWXJfnUiYjUKT
10 37i9dQZF1DXcRXFNfZr7Tp
11 37i9dQZF1DX4o1oenSJRJd
12 37i9dQZF1DXbTxeAdrVG2l
13 37i9dQZF1DX4UtSsGT1Sbe
14 37i9dQZF1DWTJ7xPn4vNaz
15 37i9dQZF1DXaKIA8E7WcJj
16 37i9dQZF1DWSV3Tk4GO2fq
17 37i9dQZF1DWTwnEm1IYyoj
18 37i9dQZF1DX2A29LI7xHn1
19 37i9dQZF1DX2RxBh64BHjQ
20 37i9dQZF1DWVA1Gq4XHa6U
21 37i9dQZF1DWY4xHQp97fN6
22 37i9dQZF1DWX3387IZmjNa
23 37i9dQZF1DWYkaDif7Ztbp
24 37i9dQZF1DX5hR0J49CmXC
25 37i9dQZF1DXan38dNVDdl4
26 37i9dQZF1DWSvKsRPPnv5o
27 37i9dQZF1DWUVpAXiEPK8P
28 37i9dQZF1DX0Tkc6ltcBfU
29 37i9dQZF1DX1YPTAhwehsC
30 37i9dQZF1DWTggY0yqBxES
31 37i9dQZF1DX0HRj9P7NxeE
32 37i9dQZF1DWT6SJaitNDax
33 37i9dQZF1DX2r0FByV5U4C
34 37i9dQZF1DXaKctwWdt4be
35 37i9dQZF1DWT2jS7NwYPVI
36 37i9dQZF1DX82GYcclJ3Ug
37 37i9dQZF1DX49jUV2NfGku
38 37i9dQZF1DX1Jg9KZ76

In [95]:
len(playListDetails['track_ids']), len(playListDetails['artist_ids'])

(69332, 24569)

In [97]:
with open("tracks.txt", "w") as f:
    for ids in playListDetails['track_ids']:
        f.write(ids + "\n")

In [99]:
with open("artists.txt", "w") as f:
    for ids in playListDetails['artist_ids']:
        f.write(ids + "\n")